# Constrained Food Recommender

In this assignment, you will implement both Content Based and Collaborative Filtering Recommenders and backtracking search (or local search) on your own

100% finished homework should contain EDA, Item and User profiles generation, Content-Based Recommender, Collaborative Filtering Recommender, and soluton to CSP problem of assigning recommendations to brekfast, lunch and dinner.

## Data loading

You will work with subset of [Academic Yelp Dataset](https://www.kaggle.com/yelp-dataset/yelp-dataset) containing list of restaurants in **yelp_business.csv** and reviews of the users in **yelp_reviews.parquet**

In [1]:
import pandas as pd

df_yelp_business = pd.read_csv("yelp_business.csv").drop(columns=["Unnamed: 0"])
df_yelp_reviews = pd.read_parquet("yelp_reviews.parquet")

# Leave only users with at least 3 reviews
users_count = df_yelp_reviews.groupby("user_id").count()[["business_id"]] 
users_to_use = users_count[users_count["business_id"] > 2]
df_yelp_reviews = df_yelp_reviews[df_yelp_reviews["user_id"].isin(users_to_use.index)]

## Exploratory data analysis

Here you will explore the data to find out what is the distribution of business categories, hours, places, user reviews, etc.

This step is needed to proceed later with item and user profiling and to clean your data if there are duplicates (e.g. duplicated reviews, the same businesses under different ids, categories tags which are highly correlated) or some artifacts not related to the main task.

(5 points)

In [2]:
df_yelp_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,404 E Green St,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...",pQeaRpvuhoEqudo3uymHIQ,"Ethnic Food, Food Trucks, Specialty Food, Impo...",Champaign,"{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",1,40.110446,-88.233073,The Empanadas House,61820,5,4.5,IL
1,4508 E Independence Blvd,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...",CsLQLiRoafpJPJSkNX2h5Q,"Food, Restaurants, Grocery, Middle Eastern",Charlotte,NaN,0,35.194894,-80.767442,Middle East Deli,28205,5,3.0,NC
2,300 John Street,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",lu7vtrp_bE9PnxWfA8g4Pg,"Japanese, Fast Food, Food Court, Restaurants",Thornhill,NaN,1,43.820492,-79.398466,Banzai Sushi,L3T 5W4,7,4.5,ON
3,"4550 East Cactus Rd, #KSFC-4","{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",vjTVxnsQEZ34XjYNS-XUpA,"Food, Pretzels, Bakeries, Fast Food, Restaurants",Phoenix,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,33.602822,-111.983533,Wetzel's Pretzels,85032,10,4.0,AZ
4,9595 W Tropicana Ave,"{'Alcohol': ""u'none'"", 'WiFi': ""u'no'"", 'GoodF...",fnZrZlqW1Z8iWgTVDfv_MA,"Mexican, Restaurants, Fast Food",Las Vegas,NaN,0,36.099738,-115.301568,Carl's Jr,89147,15,2.5,NV


In [3]:
len(df_yelp_business)

5748

In [4]:
name_indexed = df_yelp_business.groupby('name')['business_id'].nunique()
same_name_different_id = name_indexed[name_indexed != 1].index
filtered = df_yelp_business[~df_yelp_business['name'].isin(same_name_different_id)]
len(filtered)

4444

In [5]:
df_yelp_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
4,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0,6TdNDKywdbjoTkizeMce8A,4,happy day finally canes near casa yes others g...,0,UgMW8bLE0QMJDCkQ1Ax5Mg
6,Pthe4qk5xh4n-ef-9bvMSg,0,2015-11-05 23:11:05,0,ZayJ1zWyWgY9S_TRLT_y9Q,5,really good place simple decor amazing food gr...,1,aq_ZxGHiri48TUXJlpRkCQ
9,Ws8V970-mQt2X9CwCuT5zw,1,2009-10-13 04:16:41,0,z4BCgTkfNtCu4XY5Lp97ww,4,twice nice laid back tried weekend southern me...,3,jOERvhmK6_lo_XGUBPws_w
16,d4qwVw4PcN-_2mK2o1Ro1g,0,2015-02-02 06:28:00,0,bVTjZgRNq8ToxzvtiVrqMA,1,10pm super bowl sunday already closed weak won...,0,2hRe26HSCAWbFRn5WChK-Q
22,9Jo1pu0y2zU6ktiwQm6gNA,20,2016-12-04 03:15:21,19,sgTnHfeaEvyOoWX4TCgkuQ,4,coconut fish cafe fantastic five stars fish ca...,24,A0j21z2Q1HGic7jW6e9h7A


In [6]:
len(df_yelp_reviews)

79934

In [7]:
# Assuming that a review is uniquely determined by its id
df_yelp_reviews.drop_duplicates('review_id', inplace=True)
len(df_yelp_reviews)

79934

## Building recommender

First of all you should process user reviews to get the utility matrix containing ratings for users and businesses. There will be a lot of 0 in this matrix and it is better to store such matrices in the specialized data structure for sparce matrices. However, your working dataset is relatively small and we can use simple **pd.DataFrame** to proceed

In [8]:
def create_utility_matrix(reviews: pd.DataFrame, business: pd.DataFrame) -> pd.DataFrame:
    business_ids = business["business_id"].unique()
    users = reviews["user_id"].unique()
    ut_matrix = pd.DataFrame(0, columns=business_ids, index=users)
    for _, review in reviews.iterrows():
        ut_matrix.loc[review["user_id"], review["business_id"]] = review["stars"]
    # TODO: Rating normalization to (-1, 1) range (5 points)
    
    ut_matrix = ut_matrix.sub(ut_matrix.mean(axis=1), axis=0)
    
    ut_matrix = ut_matrix.sub(ut_matrix.mean(axis=0), axis=1)
    
    ut_matrix /= 4
    
    return ut_matrix

df_utility_matrix = create_utility_matrix(df_yelp_reviews, df_yelp_business)

In [9]:
df_utility_matrix.head()

,pQeaRpvuhoEqudo3uymHIQ,CsLQLiRoafpJPJSkNX2h5Q,lu7vtrp_bE9PnxWfA8g4Pg,vjTVxnsQEZ34XjYNS-XUpA,fnZrZlqW1Z8iWgTVDfv_MA,rVBPQdeayMYht4Uv_FOLHg,fhNf_sg-XzZ3e7HEVGuOZg,LoRef3ChgZKbxUio-sHgQg,Ga2Bt7xfqoggTypWD5VpoQ,xFc50drSPxXkcLvX5ygqrg,...,2SfSzEd3B7WimeZac23zhg,1dV3qNEv8nNUAX1k3qdE2w,YHCseOJ93wJh0gBcii_2qA,TJt1W9haRm2DKuoZLQ69yA,wM8QNs7uSyDqMJKjBYFPCQ,gp_bu7Ah81qaBY3M0Leffw,PUKOr5bEI87TVHjwijT1xw,zV38gkkEeJ4cVRlSWWQTfQ,H1j34TgbrVZkxeww9xlJTw,F8M0IukXQqR50IRyocRQbg
UgMW8bLE0QMJDCkQ1Ax5Mg,-0.000545,-0.000685,-0.000654,-0.000840,-0.000948,-0.000824,-0.000747,-0.000716,-0.000685,-0.000747,...,-0.010052,-0.000731,-0.000824,-0.003771,-0.000700,-0.000855,-0.000545,-0.001770,-0.002949,-0.000638
aq_ZxGHiri48TUXJlpRkCQ,0.000194,0.000055,0.000086,-0.000100,-0.000209,-0.000085,-0.000007,0.000024,0.000055,-0.000007,...,-0.009312,0.000008,-0.000085,-0.003031,0.000039,-0.000116,0.000194,-0.001031,-0.002210,0.000101
jOERvhmK6_lo_XGUBPws_w,-0.000502,-0.000641,-0.000610,-0.000796,-0.000905,-0.000781,-0.000703,-0.000672,-0.000641,-0.000703,...,-0.010008,-0.000688,-0.000781,-0.003727,-0.000657,-0.000812,-0.000502,-0.001727,-0.002905,-0.000595
2hRe26HSCAWbFRn5WChK-Q,0.000412,0.000272,0.000303,0.000117,0.000009,0.000133,0.000210,0.000241,0.000272,0.000210,...,-0.009095,0.000226,0.000133,-0.002814,0.000257,0.000102,0.000412,-0.000813,-0.001992,0.000319
A0j21z2Q1HGic7jW6e9h7A,-0.001806,-0.001946,-0.001915,-0.002101,-0.002210,-0.002086,-0.002008,-0.001977,-0.001946,-0.002008,...,-0.011313,-0.001992,-0.002086,-0.005032,-0.001961,-0.002117,-0.001806,-0.003032,-0.004210,-0.001899


## Content-Based Recommender

In [10]:
def build_business_profiles(business: pd.DataFrame) -> pd.DataFrame:
    # TODO: Feature engineering (10 points)
    
    
    features = ['is_open', 'review_count', 'stars']
    business_profiles = pd.DataFrame(0, index=business["business_id"].unique(), columns=features)
    
    categories = set()
    for categ_concat in business['categories']:
        for categ in categ_concat.split(','):
            categories.add(categ)
            
    for idx, row in business.iterrows():
        categs = row['categories'].split(',')
        for categ in categs:
            business_profiles.loc[row['business_id'], categ] = 1
    
#     business_profiles[:, features] = business[:, features]
    return business_profiles

df_business_profiles = build_business_profiles(df_yelp_business)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [11]:
df_business_profiles.head(20)

,is_open,review_count,stars,Ethnic Food,Food Trucks,Specialty Food,Imported Food,Argentine,Food,Restaurants,...,DJs,Fur Clothing,Pilates,Lighting Fixtures & Equipment,Ice Delivery,Furniture Repair,Party Equipment Rentals,Audio/Visual Equipment Rental,Furniture Rental,Tennis
pQeaRpvuhoEqudo3uymHIQ,0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CsLQLiRoafpJPJSkNX2h5Q,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lu7vtrp_bE9PnxWfA8g4Pg,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vjTVxnsQEZ34XjYNS-XUpA,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fnZrZlqW1Z8iWgTVDfv_MA,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rVBPQdeayMYht4Uv_FOLHg,0,0,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fhNf_sg-XzZ3e7HEVGuOZg,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LoRef3ChgZKbxUio-sHgQg,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ga2Bt7xfqoggTypWD5VpoQ,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xFc50drSPxXkcLvX5ygqrg,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def build_user_profiles(utility_matrix: pd.DataFrame, 
                                   business_profiles: pd.DataFrame) -> pd.DataFrame:
    # TODO: Feature aggregation (5 points)
    return pd.DataFrame(0, index=utility_matrix.index, columns=business_profiles.columns)
df_user_profiles = build_user_profiles(df_utility_matrix, df_business_profiles)

In [13]:
def predict_content_ratings(user_profiles: pd.DataFrame, business_profiles: pd.DataFrame) -> pd.DataFrame:
    # TODO: Distance based rating prediction (5 points)
    # TODO: Pointwise/Pairwase training based prediction (optional for 10 extra points)
    pass

df_content_predictions = predict_content_ratings(df_user_profiles, df_business_profiles)

## Collaborative Filtering Recommender

In [15]:
def predict_collaborative_ratings(utility_matrix: pd.DataFrame) -> pd.DataFrame:
    # TODO: User-item collaborative filtering based rating prediction (15 points)
    # TODO: UV-decomposition based rating prediction (optional for 10 extra points)
    pass

df_collaborative_predictions = predict_collaborative_ratings(df_utility_matrix)

## Evaluation

In [16]:
def score_model(utility_matrix: pd.DataFrame, predicted_utility_matrix: pd.DataFrame, model_name="model_0"):
    # TODO: Implement these by hand (each metric 1 point)
    rmse_score = 0
    map_score = 0
    coverage_score = 0
    personalization_score = 0
    intra_list_similarity_score = 0
    
    print("{} RMSE {}".format(model_name, rmse_score))
    print("MAP: {}".format(model_name, map_score))
    print("Coverage: {}".format(model_name, coverage_score))
    print("Personalization: {}".format(model_name, personalization_score))
    print("Intra-list similarity: {}".format(model_name, intra_list_similarity_score))    

score_model(df_content_predictions, df_utility_matrix, "content-based approach")
score_model(df_collaborative_predictions, df_utility_matrix, "collaborative-filtering approach")

content-based approach RMSE 0
MAP: content-based approach
Coverage: content-based approach
Personalization: content-based approach
Intra-list similarity: content-based approach
collaborative-filtering approach RMSE 0
MAP: collaborative-filtering approach
Coverage: collaborative-filtering approach
Personalization: collaborative-filtering approach
Intra-list similarity: collaborative-filtering approach


## Constraint Satisfaction Problem

We can work with the task of planing breakfast, lunch and dinner for particular user as Constraint Satisfaction Problem with

**Domain**: {all_businesses}

**Variables**: {breakfast, lunch, dinner}

**Constraints**: {constrainst regarding individual variable, or several variables at once}

We also have predicted ratings for every business and want to have personalized plan of restaurants. So we won't only satisfy our constraints, but also would like to get the maximum cumulative rating.

Take a look on prepared constraints and finish empty constraints in similar way (some of these constraints may require analytics on business data. e.g. to finish **has_coffee_constraint** you may need to determine all the categories which may include good coffee in their menu)

In [ ]:
def is_vegetarian_constraint(business_id):
    return "vegetarian" in df_yelp_business[df_yelp_business["business_id"] == business_id].categories.values[0].lower()

def has_coffee_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return False

def has_alcohol_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_at_date_at_time_meta_constraint(weekday, time, business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_at_monday_at_10am_constraint(business_id):
    return is_open_at_date_at_time("monday", "10:00", business_id)

def all_are_different_constraint(state):
    for time in ["breakfast", "dinner", "lunch"]:
        for _t in ["breakfast", "dinner", "lunch"]:
            if time == _t: continue
            business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[time]["business_id"]].categories.values[0].split(","))
            _business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[_t]["business_id"]].categories.values[0].split(","))
            if len(business_categories.intersection(_business_categories)) > \
                    len(business_categories.union(_business_categories)) // 2:
                return False
    return True

def all_are_in_the_same_city_constraint(state):
    # TODO: implement this constraint (1 point)
    return False

def all_are_in_the_same_region_meta_constraint(coordinates, threshold, state):
    # TODO: implement this constraint (1 point). Hint: use haversine distance https://pypi.org/project/haversine/
    return False

def all_are_in_test_region(state):
    return all_are_in_the_same_region_constraint({"lat": 40.110446, "lon": -115.301568}, 400, state)

def at_least_one_visited_place_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one familiar place
    return False

def at_least_one_has_coffee_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one place with coffee
    return False

In [ ]:
import random 

random.seed(42)
inspected_user = random.choice(df_yelp_reviews["user_id"].unique())

all_constraints = {
    "breakfast": [has_coffee_constraint, is_open_constraint, is_open_at_monday_at_10am_constraint],
    "lunch": [is_open_constraint],
    "dinner": [is_vegetarian_constraint, has_alcohol_constraint, is_open_constraint],
    "state": [at_least_one_has_coffee_constraint, at_least_one_visited_place_constraint, all_are_in_test_region,
             all_are_in_the_same_city_constraint, all_are_different_constraint]
}

def goal_test(state: dict, constraints: dict):
    cumulative_rating = state["breakfast"]["predicted_rating"]*state["lunch"]["predicted_rating"]*\
                        state["dinner"]["predicted_rating"]
    for k in constraints.keys():
        if k == "state":
            for c in constraints[k]:
                cumulative_rating *= c(state)
        else:
            for c in constraints[k]:
                cumulative_rating *= c(state[k]["business_id"])
    return cumulative_rating


def prepare_restaurants_plan(user_id: str, user_business_ratings: pd.DataFrame, constraints: dict):
    # TODO: assign breakfast, lunch and dinner by solving Constraint Satisfaction Problem 
    # maximizing total score and satisfying all the constraints (it should work with any configuration of constraints)
    
    # You can implement Backtracking (10) + Filtering (10) + Ordering (5) using goal_test
    # OR
    # Local Search (10) + Min-Conflicts heuristic (10) + Ordering (5) with modification of goal test to work as Min-Conflicts heuristic
    
    state = {"user_id" : user_id,
        "breakfast": 
                {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0},
            "lunch": 
    {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0},
            "dinner": {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0}}
    
    state_v = goal_test(state, constraints)

    
    return state

# TODO: replace df_utility_matrix with your best predictions
prepare_restaurants_plan(inspected_user, df_utility_matrix, all_constraints)